# Module objectives
- Generator

# Setup

Import our usual suspects

In [ ]:
import os
import pandas as pd
from dotenv import load_dotenv
from graphdatascience import GraphDataScience
from neo4j import Query, GraphDatabase, RoutingControl, Result

Load env variables

In [ ]:
load_dotenv('.env', override=True)
# Neo4j
HOST = os.getenv('ICIJ_URL')
USERNAME = os.getenv('ICIJ_USER')
PASSWORD = os.getenv('ICIJ_PASSWORD')
DATABASE = os.getenv('ICIJ_DB_NAME')

# AI
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY']=OPENAI_API_KEY

Connect to neo4j db

In [ ]:
driver = GraphDatabase.driver(
    HOST,
    auth=(USERNAME, PASSWORD)
)
driver.verify_connectivity(database=DATABASE)

# Retriver

### Test/design query

In [ ]:
person_name= "Sigmundur David Gunnlaugsson"
driver.execute_query(
    '''
    match (o:Officer{name:$name})
    return {
        person_name: o.name,
        lives_with: collect { 
            match (o)-[:REGISTERED_ADDRESS]->()<-[:REGISTERED_ADDRESS]-(p) 
            return p.name 
        },
        shares_held: collect {
            match (o)-[:REGISTERED_ADDRESS]->()<-[:REGISTERED_ADDRESS]-(p)
            with [o,p] as persons
            unwind persons as p
            match (p)-[r:SHAREHOLDER_OF]->(c)
            return {
                person_name: p.name,
                share_in_company: c.name,
                from_date: r.start_date,
                to_date: r.end_date
            }
        }
    } as result
    ''',
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df(),
    name = person_name
).head(10)

## Create agent

In [ ]:
from typing import List, Optional
from pydantic import BaseModel, Field, validator
import functools
from langchain_core.tools import tool

class Person(BaseModel):
    """
    Represents a person or legal person
    """
    person_name: str = Field(..., description="Full name of the person")

@tool
def retrieve_person_invested_in_company(person: Person) -> dict:
    """Given the full name of a person or a legal person this function retrieves information about shares held or other investements in companies"""
    name = person.person_name
    return driver.execute_query(
        ''' 
        match (o:Officer{name:$name})
        return {
            person_name: o.name,
            lives_with: collect { 
                match (o)-[:REGISTERED_ADDRESS]->()<-[:REGISTERED_ADDRESS]-(p) 
                return p.name 
            },
            shares_held: collect {
                match (o)-[:REGISTERED_ADDRESS]->()<-[:REGISTERED_ADDRESS]-(p)
                with [o,p] as persons
                unwind persons as p
                match (p)-[r:SHAREHOLDER_OF]->(c)
                return {
                    person_name: p.name,
                    share_in_company: c.name,
                    from_date: r.start_date,
                    to_date: r.end_date
                }
            }
        } as result
        ''',
        database_=DATABASE,
        name=name,
        routing_=RoutingControl.READ,
        result_transformer_= lambda r: r.single()["result"]
    )

In [ ]:
# Import langchain open ai
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from langchain_core.messages import HumanMessage

llm = ChatOpenAI(model_name="gpt-4o", temperature=0)
tools = [
    retrieve_person_invested_in_company
]
llm_with_tools = llm.bind_tools(tools)

In [ ]:

question = "What shares does Sigmundur David Gunnlaugsson own?"
messages = [HumanMessage(question)]

In [ ]:
ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

In [ ]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"retrieve_person_invested_in_company": retrieve_person_invested_in_company}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

## Generator

In [ ]:
llm_with_tools.invoke(messages)